In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('cvrm_master.csv')
from cleaning_function import *

/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_6778/1321871251.py:3: DtypeWarning: Columns (1697,1698) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cvrm_master.csv')


In [2]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data = drop_null_columns(data, 0.6)

In [3]:
numerical_cols = data.select_dtypes(include=['number']).columns
categorical_cols = data.select_dtypes(exclude=['number']).columns

# **2️⃣ Apply Mean Imputation to Numerical Data**
df_mean = data.copy()
df_mean[numerical_cols] = df_mean[numerical_cols].fillna(df_mean[numerical_cols].mean())

# **3️⃣ Apply Median Imputation to Numerical Data**
df_median = data.copy()
df_median[numerical_cols] = df_median[numerical_cols].fillna(df_median[numerical_cols].median())

df_mean.to_csv('mean_imputation.csv',index = False)
df_median.to_csv('median_imputation.csv', index = False)